# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 25 2022 8:44AM,242603,10,8303573,"Snap Medical Industries, LLC",Released
1,Jul 25 2022 8:44AM,242603,10,8319707,"Snap Medical Industries, LLC",Released
2,Jul 25 2022 8:44AM,242603,10,8319709,"Snap Medical Industries, LLC",Released
3,Jul 25 2022 8:44AM,242603,10,8319713,"Snap Medical Industries, LLC",Released
4,Jul 25 2022 8:44AM,242603,10,8319714,"Snap Medical Industries, LLC",Released
5,Jul 25 2022 8:44AM,242603,10,8320323,"Snap Medical Industries, LLC",Released
6,Jul 25 2022 8:38AM,242602,10,0085901959,ISDIN Corporation,Released
7,Jul 25 2022 8:38AM,242602,10,0085901960,ISDIN Corporation,Released
8,Jul 25 2022 8:38AM,242602,10,0085901966,ISDIN Corporation,Released
9,Jul 25 2022 8:38AM,242602,10,0085901968,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,242599,Executing,1
14,242600,Released,12
15,242601,Released,6
16,242602,Released,16
17,242603,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242599,1.0,NaN
242600,NaN,12.0
242601,NaN,6.0
242602,NaN,16.0
242603,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242576,0.0,1.0
242579,0.0,1.0
242580,0.0,1.0
242583,0.0,1.0
242590,3.0,22.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242576,0,1
242579,0,1
242580,0,1
242583,0,1
242590,3,22


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242576,0,1
1,242579,0,1
2,242580,0,1
3,242583,0,1
4,242590,3,22


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242576,,1
1,242579,,1
2,242580,,1
3,242583,,1
4,242590,3,22


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC"
6,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation
22,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc."
28,Jul 25 2022 8:34AM,242600,10,Beach Products Inc
40,Jul 25 2022 8:34AM,242599,19,"GCH Granules USA, Inc."
41,Jul 25 2022 8:30AM,242576,10,Template KITS
42,Jul 25 2022 8:30AM,242579,10,Eye Pharma Inc
43,Jul 25 2022 8:30AM,242583,10,"Nextsource Biotechnology, LLC"
44,Jul 25 2022 8:30AM,242580,10,"Senseonics, Incorporated"
45,Jul 25 2022 8:23AM,242597,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",,6
1,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation,,16
2,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc.",,6
3,Jul 25 2022 8:34AM,242600,10,Beach Products Inc,,12
4,Jul 25 2022 8:34AM,242599,19,"GCH Granules USA, Inc.",1,
5,Jul 25 2022 8:30AM,242576,10,Template KITS,,1
6,Jul 25 2022 8:30AM,242579,10,Eye Pharma Inc,,1
7,Jul 25 2022 8:30AM,242583,10,"Nextsource Biotechnology, LLC",,1
8,Jul 25 2022 8:30AM,242580,10,"Senseonics, Incorporated",,1
9,Jul 25 2022 8:23AM,242597,10,Emerginnova,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",6,
1,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation,16,
2,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc.",6,
3,Jul 25 2022 8:34AM,242600,10,Beach Products Inc,12,
4,Jul 25 2022 8:34AM,242599,19,"GCH Granules USA, Inc.",,1
5,Jul 25 2022 8:30AM,242576,10,Template KITS,1,
6,Jul 25 2022 8:30AM,242579,10,Eye Pharma Inc,1,
7,Jul 25 2022 8:30AM,242583,10,"Nextsource Biotechnology, LLC",1,
8,Jul 25 2022 8:30AM,242580,10,"Senseonics, Incorporated",1,
9,Jul 25 2022 8:23AM,242597,10,Emerginnova,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",6,
1,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation,16,
2,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc.",6,
3,Jul 25 2022 8:34AM,242600,10,Beach Products Inc,12,
4,Jul 25 2022 8:34AM,242599,19,"GCH Granules USA, Inc.",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",6.0,NaN
1,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation,16.0,NaN
2,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc.",6.0,NaN
3,Jul 25 2022 8:34AM,242600,10,Beach Products Inc,12.0,NaN
4,Jul 25 2022 8:34AM,242599,19,"GCH Granules USA, Inc.",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",6.0,0.0
1,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation,16.0,0.0
2,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc.",6.0,0.0
3,Jul 25 2022 8:34AM,242600,10,Beach Products Inc,12.0,0.0
4,Jul 25 2022 8:34AM,242599,19,"GCH Granules USA, Inc.",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2668508,77.0,2.0
16,242595,0.0,1.0
19,970372,29.0,4.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2668508,77.0,2.0
1,16,242595,0.0,1.0
2,19,970372,29.0,4.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,77.0,2.0
1,16,0.0,1.0
2,19,29.0,4.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,77.0
1,16,Released,0.0
2,19,Released,29.0
3,10,Executing,2.0
4,16,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19
Status,,,
Executing,2.0,1.0,4.0
Released,77.0,0.0,29.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19
0,Executing,2.0,1.0,4.0
1,Released,77.0,0.0,29.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19
0,Executing,2.0,1.0,4.0
1,Released,77.0,0.0,29.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()